# Advanced Regression Models in py-tidymodels

This notebook demonstrates three advanced regression techniques:

1. **MARS (Multivariate Adaptive Regression Splines)** - Automatic non-linearity detection
2. **Poisson Regression** - For count data and rare events
3. **GAM (Generalized Additive Models)** - Flexible non-parametric relationships

Each model has unique strengths for different data patterns.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from py_parsnip import mars, poisson_reg, gen_additive_mod
from py_yardstick import rmse, rsq

# Set random seed for reproducibility
np.random.seed(42)

print("Packages loaded successfully!")

## 1. MARS - Multivariate Adaptive Regression Splines

MARS is a non-parametric regression technique that:
- Automatically detects non-linear relationships
- Uses piecewise linear basis functions (hinge functions)
- Can model interactions between predictors
- Performs automatic feature selection

**Key Parameters:**
- `num_terms`: Maximum number of basis functions (default: unlimited)
- `prod_degree`: Maximum interaction degree (1=additive, 2=pairwise interactions)
- `prune_method`: How to prune the model ("backward", "none", "exhaustive")

In [ ]:
# Create non-linear data with quadratic and sinusoidal patterns
x = np.linspace(0, 10, 60)
y_mars = x**2 + np.sin(x) * 5 + np.random.normal(0, 3, 60)
data_mars = pd.DataFrame({"y": y_mars, "x": x})

# Split into train/test
train_mars = data_mars.iloc[:45]
test_mars = data_mars.iloc[45:]

print(f"Training data: {train_mars.shape[0]} observations")
print(f"Test data: {test_mars.shape[0]} observations")
print(f"\nData preview:")
print(train_mars.head())

In [ ]:
# Visualize the non-linear data
plt.figure(figsize=(10, 6))
plt.scatter(train_mars["x"], train_mars["y"], label="Training data", alpha=0.6)
plt.scatter(test_mars["x"], test_mars["y"], label="Test data", alpha=0.6, color="orange")
plt.xlabel("x")
plt.ylabel("y")
plt.title("Non-linear Data: Quadratic + Sinusoidal Pattern")
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

In [ ]:
# Fit MARS model with 15 terms and pairwise interactions
print("Fitting MARS model with 15 terms and pairwise interactions...")
spec_mars = mars(num_terms=15, prod_degree=2)
fit_mars = spec_mars.fit(train_mars, "y ~ x")

print("\nModel fitted successfully!")
print(f"Engine: {fit_mars.spec.engine}")
print(f"Model type: {fit_mars.spec.model_type}")

In [ ]:
# Make predictions
pred_mars = fit_mars.predict(test_mars)
print(f"Predictions shape: {pred_mars.shape}")
print(f"\nFirst 5 predictions:")
print(pred_mars.head())

# Calculate test set metrics
test_mars_with_pred = test_mars.copy()
test_mars_with_pred[".pred"] = pred_mars[".pred"].values

test_rmse = rmse(test_mars_with_pred, "y", ".pred")
test_rsq = rsq(test_mars_with_pred, "y", ".pred")

print(f"\nTest Set Performance:")
print(f"  RMSE: {test_rmse['.estimate'].iloc[0]:.4f}")
print(f"  R-squared: {test_rsq['.estimate'].iloc[0]:.4f}")

In [ ]:
# Extract model details
outputs_mars, basis_funcs, stats_mars = fit_mars.extract_outputs()

print(f"Model used {len(basis_funcs)} basis functions")
print(f"\nBasis functions:")
print(basis_funcs[["variable", "coefficient"]].head(10))

print(f"\nModel statistics:")
print(stats_mars)

In [ ]:
# Visualize MARS predictions
plt.figure(figsize=(12, 6))

# Plot training fit
plt.subplot(1, 2, 1)
train_outputs = outputs_mars[outputs_mars["split"] == "train"]
plt.scatter(train_outputs["x"], train_outputs["y"], label="Actuals", alpha=0.6)
plt.plot(train_outputs["x"], train_outputs[".pred"], label="MARS Fit", color="red", linewidth=2)
plt.xlabel("x")
plt.ylabel("y")
plt.title("MARS: Training Fit")
plt.legend()
plt.grid(True, alpha=0.3)

# Plot test predictions
plt.subplot(1, 2, 2)
plt.scatter(test_mars["x"], test_mars["y"], label="Actuals", alpha=0.6, color="orange")
plt.plot(test_mars["x"], pred_mars[".pred"], label="MARS Predictions", color="red", linewidth=2)
plt.xlabel("x")
plt.ylabel("y")
plt.title(f"MARS: Test Predictions (RMSE={test_rmse['.estimate'].iloc[0]:.2f})")
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 2. Poisson Regression - For Count Data

Poisson regression is designed for modeling count outcomes:
- Uses a log link function: log(E[Y]) = β₀ + β₁X₁ + β₂X₂ + ...
- Ideal for rare events, event counts, non-negative integers
- Assumes variance = mean (can be relaxed with quasi-Poisson)

**Common Applications:**
- Number of website visits per day
- Number of defects in manufacturing
- Number of insurance claims
- Epidemiology: disease counts

In [ ]:
# Create count data with two predictors
x1_pois = np.random.uniform(0, 5, 50)
x2_pois = np.random.uniform(0, 3, 50)
lambda_true = np.exp(0.3 + 0.4 * x1_pois + 0.5 * x2_pois)
counts = np.random.poisson(lambda_true)
data_pois = pd.DataFrame({"count": counts, "x1": x1_pois, "x2": x2_pois})

# Split into train/test
train_pois = data_pois.iloc[:35]
test_pois = data_pois.iloc[35:]

print(f"Training data: {train_pois.shape[0]} observations")
print(f"Test data: {test_pois.shape[0]} observations")
print(f"\nCount distribution in training data:")
print(train_pois["count"].describe())
print(f"\nData preview:")
print(train_pois.head())

In [ ]:
# Visualize the count data
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Count distribution
axes[0].hist(train_pois["count"], bins=15, edgecolor="black", alpha=0.7)
axes[0].set_xlabel("Count")
axes[0].set_ylabel("Frequency")
axes[0].set_title("Distribution of Count Data")
axes[0].grid(True, alpha=0.3)

# 3D scatter plot projection
scatter = axes[1].scatter(train_pois["x1"], train_pois["x2"], c=train_pois["count"], 
                          cmap="viridis", s=100, alpha=0.6, edgecolors="black")
axes[1].set_xlabel("x1")
axes[1].set_ylabel("x2")
axes[1].set_title("Count as Function of x1 and x2")
plt.colorbar(scatter, ax=axes[1], label="Count")

plt.tight_layout()
plt.show()

In [ ]:
# Fit Poisson GLM
print("Fitting Poisson GLM...")
spec_pois = poisson_reg()
fit_pois = spec_pois.fit(train_pois, "count ~ x1 + x2")

print("\nModel fitted successfully!")
print(f"Engine: {fit_pois.spec.engine}")
print(f"Model type: {fit_pois.spec.model_type}")

In [ ]:
# Predict with confidence intervals
pred_pois = fit_pois.predict(test_pois, type="conf_int")
print(f"Predictions with confidence intervals:")
print(pred_pois.head(10))

# Calculate test set metrics (using point predictions)
pred_pois_numeric = fit_pois.predict(test_pois, type="numeric")
test_pois_with_pred = test_pois.copy()
test_pois_with_pred[".pred"] = pred_pois_numeric[".pred"].values

test_rmse_pois = rmse(test_pois_with_pred, "count", ".pred")
print(f"\nTest Set RMSE: {test_rmse_pois['.estimate'].iloc[0]:.4f}")

In [ ]:
# Extract model details
outputs_pois, coefs_pois, stats_pois = fit_pois.extract_outputs()

print(f"Coefficients (with z-statistics):")
print(coefs_pois[["variable", "coefficient", "z_stat", "p_value"]])

print(f"\nModel statistics:")
print(stats_pois)

# Interpret coefficients
print(f"\nCoefficient interpretation:")
for idx, row in coefs_pois.iterrows():
    if row["variable"] != "Intercept":
        effect = (np.exp(row["coefficient"]) - 1) * 100
        print(f"  {row['variable']}: 1-unit increase → {effect:.1f}% change in expected count")

In [ ]:
# Visualize Poisson predictions
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Actual vs predicted
axes[0].scatter(test_pois["count"], pred_pois_numeric[".pred"], alpha=0.6, s=100)
axes[0].plot([0, test_pois["count"].max()], [0, test_pois["count"].max()], 
             "r--", linewidth=2, label="Perfect predictions")
axes[0].set_xlabel("Actual Count")
axes[0].set_ylabel("Predicted Count")
axes[0].set_title("Poisson Regression: Actual vs Predicted")
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Residuals
residuals = test_pois["count"].values - pred_pois_numeric[".pred"].values
axes[1].scatter(pred_pois_numeric[".pred"], residuals, alpha=0.6, s=100)
axes[1].axhline(y=0, color="r", linestyle="--", linewidth=2)
axes[1].set_xlabel("Predicted Count")
axes[1].set_ylabel("Residuals")
axes[1].set_title("Poisson Regression: Residual Plot")
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 3. Generalized Additive Models (GAM)

GAMs fit smooth non-parametric functions to each predictor:
- E[Y] = β₀ + f₁(X₁) + f₂(X₂) + ... where f() are smooth functions
- Automatically detects non-linear relationships
- More interpretable than black-box ML models
- Can include both smooth and linear terms

**Key Parameters:**
- `adjust_deg_free`: Degrees of freedom for smoothing (higher = more flexible)
- `select_features`: Automatic feature selection (True/False)

In [ ]:
# Create data with non-linear relationships
x1_gam = np.linspace(0, 10, 50)
x2_gam = np.linspace(0, 5, 50)
y_gam = np.sin(x1_gam) * 10 + x2_gam**2 + np.random.normal(0, 2, 50)
data_gam = pd.DataFrame({"y": y_gam, "x1": x1_gam, "x2": x2_gam})

# Split into train/test
train_gam = data_gam.iloc[:35]
test_gam = data_gam.iloc[35:]

print(f"Training data: {train_gam.shape[0]} observations")
print(f"Test data: {test_gam.shape[0]} observations")
print(f"\nData preview:")
print(train_gam.head())

In [ ]:
# Visualize the non-linear relationships
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# y vs x1 (sinusoidal pattern)
axes[0].scatter(train_gam["x1"], train_gam["y"], alpha=0.6)
axes[0].set_xlabel("x1")
axes[0].set_ylabel("y")
axes[0].set_title("Non-linear Relationship: y vs x1 (sinusoidal)")
axes[0].grid(True, alpha=0.3)

# y vs x2 (quadratic pattern)
axes[1].scatter(train_gam["x2"], train_gam["y"], alpha=0.6, color="orange")
axes[1].set_xlabel("x2")
axes[1].set_ylabel("y")
axes[1].set_title("Non-linear Relationship: y vs x2 (quadratic)")
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Fit GAM with moderate smoothing (12 splines per feature)
print("Fitting GAM with 12 splines per feature...")
spec_gam = gen_additive_mod(adjust_deg_free=12)
fit_gam = spec_gam.fit(train_gam, "y ~ x1 + x2")

print("\nModel fitted successfully!")
print(f"Engine: {fit_gam.spec.engine}")
print(f"Model type: {fit_gam.spec.model_type}")

In [ ]:
# Make predictions
pred_gam = fit_gam.predict(test_gam)
print(f"Predictions shape: {pred_gam.shape}")
print(f"\nFirst 5 predictions:")
print(pred_gam.head())

# Calculate test set metrics
test_gam_with_pred = test_gam.copy()
test_gam_with_pred[".pred"] = pred_gam[".pred"].values

test_rmse_gam = rmse(test_gam_with_pred, "y", ".pred")
test_rsq_gam = rsq(test_gam_with_pred, "y", ".pred")

print(f"\nTest Set Performance:")
print(f"  RMSE: {test_rmse_gam['.estimate'].iloc[0]:.4f}")
print(f"  R-squared: {test_rsq_gam['.estimate'].iloc[0]:.4f}")

In [ ]:
# Extract model details
outputs_gam, partial_effects, stats_gam = fit_gam.extract_outputs()

print(f"Partial effects (feature contributions):")
print(partial_effects[["feature", "effect_range", "data_range"]])

print(f"\nModel statistics:")
print(stats_gam)

In [ ]:
# Visualize GAM predictions
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Predictions for x1
train_outputs_gam = outputs_gam[outputs_gam["split"] == "train"].sort_values("x1")
axes[0].scatter(train_outputs_gam["x1"], train_outputs_gam["y"], 
                label="Actuals", alpha=0.6)
axes[0].plot(train_outputs_gam["x1"], train_outputs_gam[".pred"], 
             label="GAM Fit", color="red", linewidth=2)
axes[0].set_xlabel("x1")
axes[0].set_ylabel("y")
axes[0].set_title("GAM: Smooth Function for x1")
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Test predictions
axes[1].scatter(test_gam["y"], pred_gam[".pred"], alpha=0.6, s=100)
axes[1].plot([test_gam["y"].min(), test_gam["y"].max()], 
             [test_gam["y"].min(), test_gam["y"].max()], 
             "r--", linewidth=2, label="Perfect predictions")
axes[1].set_xlabel("Actual y")
axes[1].set_ylabel("Predicted y")
axes[1].set_title(f"GAM: Test Predictions (R²={test_rsq_gam['.estimate'].iloc[0]:.3f})")
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Model Comparison and Selection Guide

### When to Use Each Model:

#### MARS (Multivariate Adaptive Regression Splines)
**Best for:**
- Automatic non-linearity detection
- Automatic interaction detection
- When you don't know the functional form
- Piecewise linear approximations are acceptable

**Pros:**
- Automatically finds knots and interactions
- Built-in feature selection
- Interpretable piecewise linear functions
- Handles missing data well

**Cons:**
- Can overfit with too many terms
- Less smooth than GAMs
- Harder to interpret with many basis functions

#### Poisson Regression
**Best for:**
- Count data (non-negative integers)
- Rare events
- When variance approximately equals mean
- Epidemiology, quality control, website analytics

**Pros:**
- Specifically designed for count data
- Interpretable coefficients (multiplicative effects)
- Standard statistical inference (p-values, confidence intervals)
- Handles zero counts naturally

**Cons:**
- Assumes mean = variance (can use quasi-Poisson if violated)
- Limited to count outcomes
- May not fit overdispersed data well

#### GAM (Generalized Additive Model)
**Best for:**
- Smooth non-parametric relationships
- When you need interpretable non-linear effects
- Visualizing how each predictor affects outcome
- When flexibility is needed but interpretability matters

**Pros:**
- Very flexible smooth functions
- More interpretable than black-box ML
- Can visualize partial effects
- Works with different distributions (Gaussian, Poisson, etc.)

**Cons:**
- Doesn't capture interactions automatically
- Can be slow with large datasets
- Requires tuning smoothing parameters
- Assumes additive structure

### Quick Selection Guide:

```
Data Type: Count data → Use Poisson Regression

Need smooth curves + interpretability → Use GAM

Unknown functional form + want automatic interactions → Use MARS

Need maximum flexibility + interactions → Use MARS or ensemble methods

Need statistical inference (p-values) → Use Poisson or GAM
```

## Summary

This notebook demonstrated three advanced regression techniques in py-tidymodels:

1. **MARS** - Automatic non-linearity and interaction detection with piecewise linear functions
2. **Poisson Regression** - Statistical modeling of count data with interpretable coefficients
3. **GAM** - Flexible smooth non-parametric functions with interpretable partial effects

All three models:
- Follow the tidymodels workflow (spec → fit → predict)
- Support the three-DataFrame output structure
- Work with py_workflows for preprocessing
- Can be tuned with py_tune
- Provide comprehensive model diagnostics

Choose the model that best matches your:
- Data type (continuous vs count)
- Need for interpretability
- Type of non-linearity expected
- Whether interactions are important